In [4]:
from utils import TTS

#os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [5]:
import torch 
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4050 Laptop GPU'

In [6]:
im_cp = TTS()

INIT


In [ ]:
#im_cp.subtitles_video_with_display('sample3.mp4',is_url=False,url='https://youtu.be/o-GdKXiM0gU',display=True,languag='en',save=False)

In [7]:
im_cp.translate = False
im_cp.translator.set_source_language ('tur_Latn')
im_cp.translator.set_target_language ('eng_Latn')


In [10]:
im_cp.subtitles_video('sample_tr_dize.mp4',chunk_length=60,save_txt=True,language='tr')

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 5.77 GiB of which 13.31 MiB is free. Including non-PyTorch memory, this process has 5.73 GiB memory in use. Of the allocated memory 5.35 GiB is allocated by PyTorch, and 301.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from utils import write_subs_to_video
write_subs_to_video(video_fn="sample_tr_dize.mp4",subs_fn="sample_tr_dize.srt",output_path='sample_en_translated.mp4',font_sz=20,max_length=60)

In [ ]:
with open("/home/kriselda/Downloads/arrival.txt", 'r', encoding="utf-8") as f:

    ARRIVAL = f.read()


In [ ]:
with open("full_episode.txt", "r") as srtfile:
    NANNY = srtfile.read()


In [ ]:
from utils import get_text_from_srt
NICEGUYS=get_text_from_srt('theniceguys.srt')
OTHER = get_text_from_srt('theotherguys.srt')
B99 = get_text_from_srt('b99.srt')
LO = get_text_from_srt('lawandorder.srt')

In [ ]:
from utils import divide_text_into_sentence_batches
input_batches = divide_text_into_sentence_batches(OTHER,max_tokens_per_batch=900)

In [ ]:

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define input text and query
input_text = """
Abstractive summarization is a challenging task in natural language processing. 
It involves generating a summary that captures the main ideas of the input text 
in a concise and coherent way. Unlike extractive summarization, which selects 
sentences from the input text, abstractive summarization involves generating 
new sentences to convey the essence of the text. Pre-trained models like GPT-2 
have shown promising results in abstractive summarization tasks.
"""
query = "write  summarization"

# Tokenize input text and query
input_ids = tokenizer.encode(input_batches[0], return_tensors="pt", max_length=1024, truncation=True)
query_ids = tokenizer.encode(query, return_tensors="pt", max_length=50, truncation=True)

# Generate summary by conditioning on the concatenated input
input_ids = torch.cat([query_ids, input_ids], dim=1)
output = model.generate(input_ids, max_length=150, min_length=40, num_beams=4, early_stopping=True)

# Decode and print the generated summary
summary_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Input Text:")
print(input_text)
print("\nQuery:")
print(query)
print("\nGenerated Summary:")
print(summary_text)

In [ ]:
#flan is not working for summarization
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
for input_text_n in input_batches:
    input_ids = tokenizer('Summarize : '+ input_text_n, return_tensors="pt").input_ids

    outputs = model.generate(input_ids)
    print(tokenizer.decode(outputs[0]))

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
# philschmid/bart-large-cnn-samsum is better that flan-t5 but not acceptable
# Load pre-trained BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('philschmid/flan-t5-base-samsum')
model = BartForConditionalGeneration.from_pretrained('philschmid/bart-large-cnn-samsum')

# Input text to be summarized
for input_text_n in input_batches:

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=200, min_length=50, num_beams=4, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Print the generated summary
    print("Generated Summary:")
    print(summary)



In [ ]:

from transformers import Conversation, ConversationPipeline, GPT2ForConversation, GPT2Tokenizer

# Load pretrained model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForConversation.from_pretrained(model_name)

# Define a conversation
conversation = Conversation("Hello, how can I assist you today?")

# Define a query
query = f"Summerize in a few sentences: {input_batches[0]}"

# Add query to the conversation
conversation.add_user_input(query)

# Append the conversation history to the input_ids
input_ids = tokenizer.encode(conversation, return_tensors="pt")

# Generate a response
response_ids = model.generate(input_ids)

# Decode the response and print it
response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)
print("Response:", response_text)



In [ ]:
from transformers import pipeline
from utils import divide_text_into_sentence_batches

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

batches = divide_text_into_sentence_batches(B99,max_tokens_per_batch=900)
for i in range(len(batches)):

    print(summarizer(batches[i], max_length=130, min_length=30, do_sample=False))


In [ ]:
len(LO[15000:].split(' '))

In [ ]:
LO[15000:]

In [ ]:
from utils import get_text_from_srt
NANNY=get_text_from_srt("nanny.srt")

In [ ]:
len(B99.split(' '))

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cpu" # the device to load the model onto
qwen_mode_name = "Qwen/Qwen1.5-0.5B-Chat"

model = AutoModelForCausalLM.from_pretrained(
    qwen_mode_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)  
tokenizer = AutoTokenizer.from_pretrained(qwen_mode_name)

prompt = f"Summarize as a teaser synopsis:{NANNY}"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
response

In [ ]:
from utils import get_text_from_srt
NANNY=get_text_from_srt('b99s04e05.srt')

In [ ]:
from utils import SummarizerQwen
sq = SummarizerQwen(qwen_model_name="Qwen/Qwen1.5-0.5B-Chat",device='cpu')

In [ ]:
sq.summarize_gamma()

In [ ]:
task = "summarize as a teaser synopsis in 3 sentences"
prompt_system = "You are a helpful assistant."

sq.summarize(task=task,input_text=NANNY,prompt_sys=prompt_system)

In [ ]:
len(NANNY)

In [ ]:
# import moviepy.editor as mp

# video_path = 'sample_with_subs.mp4'
# audio_path = 'whole.mp3'
# output_path = 'sample3example.mp4'

# video = mp.VideoFileClip(video_path)
# audio = mp.AudioFileClip(audio_path)

# video = video.set_audio(audio)

# video.write_videofile(output_path)